In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install autogluon

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
column_names = ['age', 'sex','cp','trestbps','chol','fbs','resteccg','thalach','exange','oldpeak','slope','ca','thall', 'num']
df = pd.read_csv('/content/drive/MyDrive/deneme1.data',header=None ,names=column_names ,na_values='?')

x=df.drop(['num' ,'ca','thall','slope'],axis=1).values         #Dataframe değil de numpy array formatında olması için sonuna .values ekledim
y=df['num']

df

,age,sex,cp,trestbps,chol,fbs,resteccg,thalach,exange,oldpeak,slope,ca,thall,num
0,63,1,1,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67,1,4,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67,1,4,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37,1,3,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41,0,2,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,0,4,127.0,333.0,1.0,1.0,154.0,0.0,0.0,NaN,NaN,NaN,1
916,62,1,1,NaN,139.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0
917,55,1,4,122.0,223.0,1.0,1.0,100.0,0.0,0.0,NaN,NaN,6.0,2
918,58,1,4,NaN,385.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
print(df.isnull().sum())

age           0
sex           0
cp            0
trestbps     59
chol         30
fbs          90
resteccg      2
thalach      55
exange       55
oldpeak      62
slope       309
ca          611
thall       486
num           0
dtype: int64


Taking care of missing data

In [5]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan , strategy='mean')
imputer.fit(x[: ,3:10])                                              #bu adımda sadece ortalama değer hesaplanır
x[: ,3:10]=imputer.transform(x[: ,3:10])                             #Eksik değerleri doldurulan dataset, dönüştürülmüş dataset olarak döndürülür.  SimpleImputer kullanırken np.array işlemi gerekli değildir.  doğrudan dönüştürülmüş veriyi döndürdüğü için, bu dönüştürülmüş veriyi doğrudan kullanabilirsiniz.
x

array([[ 63.        ,   1.        ,   1.        , ..., 150.        ,
          0.        ,   2.3       ],
       [ 67.        ,   1.        ,   4.        , ..., 108.        ,
          1.        ,   1.5       ],
       [ 67.        ,   1.        ,   4.        , ..., 129.        ,
          1.        ,   2.6       ],
       ...,
       [ 55.        ,   1.        ,   4.        , ..., 100.        ,
          0.        ,   0.        ],
       [ 58.        ,   1.        ,   4.        , ..., 137.54566474,
          0.38959538,   0.87878788],
       [ 62.        ,   1.        ,   2.        , ...,  93.        ,
          1.        ,   0.        ]])

In [6]:
from plotly.express import scatter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# PCA modeli oluşturma
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x_scaled)
pca_df = pd.DataFrame(x_pca, columns=["x", "y"])

y_df=pd.DataFrame(y, columns=["num"])

plot_df = pd.concat(objs=[y_df, pca_df], axis=1)

scatter(data_frame=plot_df, x='x', y='y', color='num', height=500)

In [7]:
class_counts = df['num'].value_counts()
print(class_counts)

num
0    411
1    265
2    109
3    107
4     28
Name: count, dtype: int64


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)                               #split den sonda yapsan daha iyi olabilir, y ye de uygulaman gerekeilir

# Veri setini bölme
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=100)
# Random Forest Classifier modelini oluşturma ve eğitme
class_weights = {0:1, 1:1, 2:2, 3:2, 4:8}
rf_classifier =RandomForestClassifier(class_weight=class_weights, random_state=100)
rf_classifier.fit(x_train, y_train)

# Test verisi üzerinde modelin performansını değerlendirme
y_pred = rf_classifier.predict(x_test)

# Sınıflandırma raporunu oluşturma
report = classification_report(y_test, y_pred)
print("RandomForestClassifier Sınıflandırma Raporu:")
print(report)

RandomForestClassifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.72      0.87      0.78        84
           1       0.55      0.53      0.54        59
           2       0.50      0.24      0.32        21
           3       0.23      0.18      0.20        17
           4       0.00      0.00      0.00         3

    accuracy                           0.61       184
   macro avg       0.40      0.36      0.37       184
weighted avg       0.58      0.61      0.59       184



WUHUUUUUU ACC YÜKSELDİ, SMOTE:

In [ ]:
!pip install scikit-learn==0.14.1

In [ ]:
!pip install -U scikit-learn imbalanced-learn

In [10]:
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

adasyn = ADASYN(random_state=42)
x_resampled, y_resampled = adasyn.fit_resample(x_scaled, y)


adasyn = ADASYN(random_state=42)
x_resampled_ADASYN, y_resampled_ADASYN = adasyn.fit_resample(x_scaled, y)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

x_train_A, x_test_A, y_train_A, y_test_A = train_test_split(x_resampled_ADASYN, y_resampled_ADASYN, test_size=0.2, random_state=100, stratify=y_resampled_ADASYN)

class_weights = {0:1, 1:1, 2:2, 3:2, 4:8}
rf_classifier =RandomForestClassifier(class_weight=class_weights, random_state=100)
rf_classifier.fit(x_train_A, y_train_A)

y_pred = rf_classifier.predict(x_test_A)

report = classification_report(y_test_A, y_pred)
print("RandomForestClassifier Sınıflandırma Raporu:")
print(report)

RandomForestClassifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.84      0.79      0.82        82
           1       0.69      0.65      0.67        89
           2       0.82      0.85      0.83        85
           3       0.84      0.85      0.84        80
           4       0.92      0.99      0.95        83

    accuracy                           0.82       419
   macro avg       0.82      0.83      0.82       419
weighted avg       0.82      0.82      0.82       419



In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(RandomForestClassifier(class_weight=class_weights, random_state=100), param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor:", grid_search.best_score_)

best_rf_classifier = grid_search.best_estimator_

y_pred_best = best_rf_classifier.predict(x_test)

report_best = classification_report(y_test, y_pred_best)
print("En iyi Random Forest Classifier Sınıflandırma Raporu:")
print(report_best)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning:


405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    result = {}
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1344, in wrapper
    class MultiOutputMixin:
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 95, in 

En iyi parametreler: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
En iyi skor: 0.5652325795182938
En iyi Random Forest Classifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.72      0.85      0.78        84
           1       0.62      0.53      0.57        59
           2       0.50      0.38      0.43        21
           3       0.20      0.18      0.19        17
           4       0.00      0.00      0.00         3

    accuracy                           0.61       184
   macro avg       0.41      0.39      0.39       184
weighted avg       0.60      0.61      0.60       184



In [15]:
from imblearn.over_sampling import SMOTE

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_scaled, y)
x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=100, stratify=y_resampled)

In [16]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(x_train_s, y_train_s)

y_pred = knn_model.predict(x_test_s)

report = classification_report(y_test_s, y_pred)
print("KNeighborsClassifier Sınıflandırma Raporu:")
print(report)

KNeighborsClassifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.78      0.64      0.70        83
           1       0.62      0.41      0.50        82
           2       0.73      0.88      0.80        82
           3       0.73      0.79      0.76        82
           4       0.82      1.00      0.90        82

    accuracy                           0.74       411
   macro avg       0.74      0.74      0.73       411
weighted avg       0.74      0.74      0.73       411



In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [3, 5, 7, 9],  # Komşu sayısı için aralık
    'weights': ['uniform', 'distance'],  # Ağırlık fonksiyonu için seçenekler
    'metric': ['euclidean', 'manhattan']  # Uzaklık metriği seçenekleri
}

knn_model = KNeighborsClassifier()

grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid, cv=5)

grid_search.fit(x_train_s, y_train_s)

print("En iyi parametreler:", grid_search.best_params_)
print("En iyi çapraz doğrulama skoru:", grid_search.best_score_)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(x_test_s)

report = classification_report(y_test_s, y_pred)
print("KNeighborsClassifier Sınıflandırma Raporu:")
print(report)

En iyi parametreler: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
En iyi çapraz doğrulama skoru: 0.8187504633404996
KNeighborsClassifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.83      0.65      0.73        83
           1       0.75      0.68      0.71        82
           2       0.85      0.94      0.89        82
           3       0.84      0.93      0.88        82
           4       0.91      1.00      0.95        82

    accuracy                           0.84       411
   macro avg       0.84      0.84      0.83       411
weighted avg       0.84      0.84      0.83       411



In [18]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report


svm_model = SVC()  # Lineer çekirdek kullanıyoruz
svm_model.fit(x_train, y_train)   # eğitiyotuz verilerimizle

y_pred = svm_model.predict(x_test)

report = classification_report(y_test, y_pred)
print("SVC Sınıflandırma Raporu:")
print(report)

SVC Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.72      0.88      0.79        84
           1       0.50      0.59      0.54        59
           2       0.50      0.10      0.16        21
           3       0.14      0.06      0.08        17
           4       0.00      0.00      0.00         3

    accuracy                           0.61       184
   macro avg       0.37      0.33      0.32       184
weighted avg       0.56      0.61      0.56       184



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

decision_tree_model = DecisionTreeClassifier(class_weight='balanced')

decision_tree_model.fit(x_train, y_train)

y_pred = decision_tree_model.predict(x_test)

report = classification_report(y_test, y_pred)
print("DecisionTreeClassifier Sınıflandırma Raporu:")
print(report)

DecisionTreeClassifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.69      0.74      0.71        84
           1       0.57      0.46      0.51        59
           2       0.43      0.29      0.34        21
           3       0.17      0.24      0.20        17
           4       0.11      0.33      0.17         3

    accuracy                           0.54       184
   macro avg       0.39      0.41      0.39       184
weighted avg       0.56      0.54      0.55       184



In [20]:
from sklearn.ensemble import GradientBoostingClassifier

gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(x_train, y_train)

y_pred = gb_classifier.predict(x_test)

report = classification_report(y_test, y_pred)
print("GradientBoostingClassifier Sınıflandırma Raporu:")
print(report)

GradientBoostingClassifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.68      0.87      0.76        84
           1       0.55      0.47      0.51        59
           2       0.33      0.19      0.24        21
           3       0.23      0.18      0.20        17
           4       0.00      0.00      0.00         3

    accuracy                           0.59       184
   macro avg       0.36      0.34      0.34       184
weighted avg       0.54      0.59      0.56       184



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [21]:
x_train_df = pd.DataFrame(x_train)
y_train_df = pd.DataFrame(y_train)
merged_df = pd.concat([x_train_df.reset_index(drop=True), y_train_df.reset_index(drop=True)], axis=1)

In [22]:
import pandas as pd
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(label='num').fit(train_data=merged_df)

x_test_df = pd.DataFrame(x_test)
y_pred_autogluon = predictor.predict(x_test_df)

report_autogluon = classification_report(y_test, y_pred_autogluon)
print("AutoGluon Sınıflandırma Raporu:")
print(report_autogluon)

No path specified. Models will be saved in: "AutogluonModels/ag-20240501_214054"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240501_214054"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
C

AutoGluon Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.68      0.92      0.78        84
           1       0.56      0.49      0.52        59
           2       0.20      0.10      0.13        21
           3       0.44      0.24      0.31        17
           4       0.00      0.00      0.00         3

    accuracy                           0.61       184
   macro avg       0.38      0.35      0.35       184
weighted avg       0.55      0.61      0.57       184



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Çoğaltılmamış zaten olan verilerimizle

In [23]:
x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(x, y, test_size=0.2, random_state=100, stratify=y)
x_train_r_df = pd.DataFrame(x_train_r)
y_train_r_df = pd.DataFrame(y_train_r)
merged_df_withoutSTOMA = pd.concat([x_train_r_df.reset_index(drop=True), y_train_r_df.reset_index(drop=True)], axis=1)

predictor_withoutSTOMA = TabularPredictor(label='num').fit(train_data=merged_df_withoutSTOMA)

x_test_df = pd.DataFrame(x_test_r)
y_pred_autogluon = predictor_withoutSTOMA.predict(x_test_df)

report_autogluon = classification_report(y_test_r, y_pred_autogluon)
print("AutoGluon Sınıflandırma Raporu:")
print(report_autogluon)

No path specified. Models will be saved in: "AutogluonModels/ag-20240501_214213"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240501_214213"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
C

AutoGluon Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.71      0.90      0.80        82
           1       0.51      0.58      0.54        53
           2       0.00      0.00      0.00        22
           3       0.42      0.24      0.30        21
           4       0.00      0.00      0.00         6

    accuracy                           0.60       184
   macro avg       0.33      0.35      0.33       184
weighted avg       0.51      0.60      0.55       184



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [24]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Farklı sınıflandırıcıları tanımla
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
et = ExtraTreesClassifier(n_estimators=100, random_state=42)
knn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan')  # En iyi parametreler
svc = SVC(kernel='linear', probability=True)  # probability=True, predict_proba yöntemini kullanmak için gereklidir

# Ensemble modelini oluştur
ensemble_model = VotingClassifier(estimators=[
    ('rfc', rfc),
    ('et', et),
    ('knn', knn),
    ('svc', svc)
], voting='hard')

# Ensemble modelini eğit
ensemble_model.fit(x_train, y_train)

# Ensemble modeliyle tahmin yap
y_pred = ensemble_model.predict(x_test)

# Doğruluk skorunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Model Doğruluk Skoru:", accuracy)


Ensemble Model Doğruluk Skoru: 0.592391304347826


In [ ]:
!pip install pycaret

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import setup, compare_models

setup_cls = setup(data=merged_df, target='num')

best_cls_model = compare_models()


,Description,Value
0,Session id,2420
1,Target,num
2,Target type,Multiclass
3,Original data shape,"(1673, 11)"
4,Transformed data shape,"(1673, 11)"
5,Transformed train set shape,"(1171, 11)"
6,Transformed test set shape,"(502, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8002,0.9515,0.8002,0.8020,0.7973,0.7502,0.7519,0.2720
catboost,CatBoost Classifier,0.7634,0.9384,0.7634,0.7639,0.7602,0.7043,0.7059,15.5910
rf,Random Forest Classifier,0.7557,0.9383,0.7557,0.7543,0.7515,0.6947,0.6964,0.4790
lightgbm,Light Gradient Boosting Machine,0.7318,0.9267,0.7318,0.7307,0.7289,0.6647,0.6658,4.1050
xgboost,Extreme Gradient Boosting,0.7190,0.9217,0.7190,0.7221,0.7168,0.6487,0.6505,0.4620
knn,K Neighbors Classifier,0.6730,0.8996,0.6730,0.6715,0.6646,0.5911,0.5944,0.1230
gbc,Gradient Boosting Classifier,0.6559,0.0000,0.6559,0.6543,0.6493,0.5696,0.5721,1.6830
dt,Decision Tree Classifier,0.6192,0.7615,0.6192,0.6261,0.6171,0.5238,0.5262,0.0820
qda,Quadratic Discriminant Analysis,0.5107,0.0000,0.5107,0.4957,0.4945,0.3890,0.3926,0.0580
lda,Linear Discriminant Analysis,0.4800,0.0000,0.4800,0.4755,0.4724,0.3496,0.3515,0.0330


Processing:   0%|          | 0/69 [00:00<?, ?it/s]